# Build GA 2020 Source Data

This notebook accepts the Census 2020 DHC data and polling location data to build a csv to be used with scip.  Clone this notebook for new states or years.

## Census Data:

### 2020 Redistricting [P4 group](https://api.census.gov/data/2010/dec/sf1/groups/P4.html)
[HISPANIC OR LATINO, AND NOT HISPANIC OR LATINO BY RACE FOR THE POPULATION 18 YEARS AND OVER](https://data.census.gov/table?g=050XX00US13135$1000000&d=DEC+Redistricting+Data+(PL+94-171)&tid=DECENNIALPL2020.P4)
1. Select Geography:
   1. Filter for Geography -> Blocks -> State -> County Name, State -> All Blocks within County Name, State
   1. If asked to select table vintage, select 2020;  DEC Redistricting Data (PL-94-171)

Columns we want from P4:
* Total population
* Total hispanic
* Total non-hispanic

### 2020 Redistricting [P3 group](https://api.census.gov/data/2010/dec/sf1/groups/P3.html)
Data source that includes Race and geography: 
[RACE FOR THE POPULATION 18 YEARS AND OVER](https://data.census.gov/table?q=P3:+RACE+FOR+THE+POPULATION+18+YEARS+AND+OVER&tid=DECENNIALPL2020.P3)
1. Select Geography:
   1. Filter for Geography -> Blocks -> State -> County Name, State -> All Blocks within County Name, State
   1. If asked to select table vintage, select 2020;  DEC Redistricting Data (PL-94-171)

Colums we want from P3:
* White alone
* Black or African American alone
* American Indian And Alaska Native alone
* Asian alone
* Native Hawaiian and Other Pacific Islander alone
* Some Other Race alone
* Two or More Races


### 2020 Tiger/Line Shapefiles: Blocks (2020) 
Source Data https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341
* Scroll down to FTP Archiv by State
* Click on desired States
* Click on desired FIPS Code for the County
* Download that *_tablock20.zip and the *_bg20.zip files

Documentation: https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line/2020.html

Columns we want from blocks:
* GEO_ID - obtained by converting values in GEOID20 column and preppending "1000000US", e.g. 131510703153004 -> 1000000US131510703153004
* geometry - the polygon of the block
* INTPTLAT20 - latitude of block centroid
* INTPTLON20 - longitude of block centroid

**Susama:** please verify the above

# Instructions


## Redistricting source data:
1. Download desired Census Data
   1. P4 zip file from Census
   1. P3 zip file from Census
   1. Recommend downloading one county's worth of data at a time.
1. Create a directory to datasets/census/redistricting/County_Name_ST **Chad**: This is a change to  your folder structure. I want to indicate geography in the name
   1. E.g datasets/census/redistricting/Gwinnett_GA
   1. Key files: DECENNIALPL2020.P3-Data.csv; DECENNIALPL2020.P4-Data.csv
   1. Note, this requires one to filter for an individual county from the census.
1. Unzip the downloaded file P11 file zip to the directory 

## Tiger/Line Shapefile
1. Download desired Tiger/Line file zip file from Census
1. Create a directory to datasets/census/tiger/County_Name_St e.g. datasets/census/tiger/Gwinnett_GA
1. Unzip the downloaded Tiger/Line zip file to the directory 
    
## Run cells
1. Update constants such as P11_SOURCE_FILE, P3_SOURCE_FILE, BLOCK_SOURCE_FILE as needed
1. Run each cell


In [1]:
# Load the P11 csv source data into a data frame and filter out unneeded columns

import pandas as pd  
import numpy as np  
from haversine import haversine, Unit
import geopandas as gpd

P4_SOURCE_FILE = 'datasets/census/redistricting/Gwinnett_GA/DECENNIALPL2020.P4-Data.csv'
P3_SOURCE_FILE = 'datasets/census/redistricting/Gwinnett_GA/DECENNIALPL2020.P3-Data.csv'

# The column in the P4 and P4data that contains the GEO id.  This will be used later to join
# against the Block Shape File
P4_GEOID = 'GEO_ID'
P3_GEOID = 'GEO_ID'

# Prefix to add to Shape files to join them with this P11.  Note this
# needs to match the prefix found in GEO_ID output from this cell.
GEO_ID_PREFIX = '1000000US'

PL_P4_COLUMNS = [
    P4_GEOID,
    'NAME',
    'P4_001N', # Total population
    'P4_002N', # Total hispanic
    'P4_003N', # Total non-hispanic
]

PL_P3_COLUMNS = [
    P3_GEOID,
    'NAME',
    'P3_001N', # Total population
    'P3_002N', # White alone
    'P3_003N', # Black or African American alone
    'P3_004N', # American Indian or Alaska Native alone
    'P3_005N', # Asian alone
    'P3_006N', # Native Hawaiian and Other Pacific Islander alone
    'P3_007N', # Some other race alone 
    'P3_008N', # Two or More Races   
]

pd.set_option('display.max_columns', None)

#print('DHC P11 File')
#print(f'  Source {P11_SOURCE_FILE}')

p4_df = pd.read_csv(
    P4_SOURCE_FILE,
    header=[0,1], # DHC files have two headers rows when exported to CSV - tell pandas to only take top one
    low_memory=False, # files are too big, set this to False to prevent errors
    # nrows=10, # limit rows loaded - testing purposes only
)

p3_df = pd.read_csv(
    P3_SOURCE_FILE,
    header=[0,1], # DHC files have two headers rows when exported to CSV - tell pandas to take top one
    low_memory=False, # files are too big, set this to False to prevent errors
    # nrows=10, # limit rows loaded - testing purposes only
)


# Filter out the un-needed columns and keep only one header
p4_df = p4_df[PL_P4_COLUMNS]
p4_df.columns = p4_df.columns=[multicols[0] for multicols in p4_df.columns]

p3_df = p3_df[PL_P3_COLUMNS]
p3_df.columns = p3_df.columns=[multicols[0] for multicols in p3_df.columns]

(p3_df.shape, p4_df.shape)

((6842, 10), (6842, 5))

In [2]:
#Merge the data sets to get a joint demographics set
demographics = p4_df.merge(p3_df, left_on=['GEO_ID', 'NAME'], right_on=['GEO_ID', 'NAME'],how = 'outer')

#Consistency check for the data pull
demographics['Pop_diff'] = demographics.P4_001N- demographics.P3_001N
if demographics.loc[demographics.Pop_diff != 0].shape[0]!=0:
    raise ValueError('Populations different in P3 and P4. Are both pulled from the voting age universe?')

#Change column names
demographics.drop(['P4_001N', 'Pop_diff'], axis =1, inplace = True)
demographics = demographics.rename(columns = {'P4_002N': 'hispanic', 'P4_003N':'non-hispanic', 'P3_001N':'population', 'P3_002N':'white', 'P3_003N':'black', 
                      'P3_004N':'native', 'P3_005N':'asian', 'P3_006N':'pacific_islander', 'P3_007N':'other', 'P3_008N':'multiple_races'})
#Note, Hispanic is an ethnicity, not a race. The P4 columns add to the total population. The P3 columns add to the total population
demographics.columns

Index(['GEO_ID', 'NAME', 'hispanic', 'non-hispanic', 'population', 'white',
       'black', 'native', 'asian', 'pacific_islander', 'other',
       'multiple_races'],
      dtype='object')

In [3]:
# Load the census block shape file using geopandas and filter out unneeded columns

BLOCK_SOURCE_FILE = 'datasets/census/tiger/Gwinnett_GA/tl_2020_13135_tabblock20.shp'

BLOCK_SHAPE_COLS = [
    'GEOID20',
    'INTPTLAT20', 
    'INTPTLON20',
]

blocks_gdf = gpd.read_file(BLOCK_SOURCE_FILE)
blocks_gdf = blocks_gdf[BLOCK_SHAPE_COLS]


In [4]:
# Combine the PL demographic data with the block shape file using a join on GEO IDs
#drop geo_id_prefix
demographics['GEO_ID'] = demographics['GEO_ID'].str.replace(GEO_ID_PREFIX, '')


combined_df = demographics.merge(blocks_gdf, left_on='GEO_ID', right_on = 'GEOID20',how='left')

#make lat/ long floats
combined_df.INTPTLAT20 = combined_df.INTPTLAT20.astype(float)
combined_df.INTPTLON20 = combined_df.INTPTLON20.astype(float)

display(combined_df.shape)
display(demographics.head())
#make combined_df into geopandas objects

#combined_df = gpd.GeoDataFrame(
#    combined_df, geometry=gpd.points_from_xy(combined_df.INTPTLON20, combined_df.INTPTLAT20), crs="ESRI:103263")


(6842, 15)

,GEO_ID,NAME,hispanic,non-hispanic,population,white,black,native,asian,pacific_islander,other,multiple_races
0,131350501051000,"Block 1000, Block Group 1, Census Tract 501.05...",0,2,2,2,0,0,0,2,0,0
1,131350501051001,"Block 1001, Block Group 1, Census Tract 501.05...",1,10,11,11,9,1,1,0,0,0
2,131350501051002,"Block 1002, Block Group 1, Census Tract 501.05...",1,9,10,10,5,3,2,0,0,0
3,131350501051003,"Block 1003, Block Group 1, Census Tract 501.05...",0,0,0,0,0,0,0,0,0,0
4,131350501051004,"Block 1004, Block Group 1, Census Tract 501.05...",0,0,0,0,0,0,0,0,0,0


In [8]:
# Load the polling location data 

LOCATION_SOURCE_FILE = 'datasets/polling/Gwinnett_GA/Gwinnett_GA_locations_only.csv'

LOCATIONS_COLS = [
    'Location',
    'Address',
    'Location type', 
    'Lat, Long',
]

locations = pd.read_csv(LOCATION_SOURCE_FILE)
locations = locations[LOCATIONS_COLS]
#add a destination type column
locations['dest_type'] = 'polling'
locations['dest_type'].mask(locations['Location type'].str.contains('Potential'), 'potential', inplace=True)

#change the lat, long into two columns
locations[['Latitude', 'Longitude']] = locations['Lat, Long'].str.split(pat = ', ', expand=True).astype(float)
locations.drop(['Lat, Long'], axis =1, inplace = True)

# Load the census block shape file using geopandas and filter out unneeded columns

BLOCK_GROUP_SOURCE_FILE = 'datasets/census/tiger/Gwinnett_GA/tl_2020_13135_bg20.shp'

BLOCK_GROUP_SHAPE_COLS = [
    'GEOID20',
    'INTPTLAT20', 
    'INTPTLON20',
]

blockgroup_gdf = gpd.read_file(BLOCK_GROUP_SOURCE_FILE)
blockgroup_gdf = blockgroup_gdf[BLOCK_GROUP_SHAPE_COLS]

#rename to match locations data 
blockgroup_gdf = blockgroup_gdf.rename(columns = {'GEOID20': 'Location', 'INTPTLAT20':'Latitude', 'INTPTLON20':'Longitude'})
blockgroup_gdf['Address'] = None
blockgroup_gdf['Location type'] = 'bg_centroid'
blockgroup_gdf['dest_type'] = 'bg_centroid'


#Concatenate
all_locations = pd.concat([locations, blockgroup_gdf])

#Lat and Long current mix of string and geometry. Make them all floats
all_locations['Latitude'] = pd.to_numeric(all_locations['Latitude'])
all_locations['Longitude'] = pd.to_numeric(all_locations['Longitude'])

if len(all_locations.Location) != len(set(all_locations.Location)):
    raise ValueError('Non-unique names in Location column. This will cause errors later.')

#make locations into geopandas objects
#all_locations = gpd.GeoDataFrame(
#    all_locations, geometry=gpd.points_from_xy(all_locations.Longitude, all_locations.Latitude), #crs="ESRI:103263"
#).to_crs("ESRI:103263")
#print(all_locations.crs.axis_info[0].unit_name)
display(all_locations.columns)
display(all_locations.head())
display(blockgroup_gdf.columns)


Index(['Location', 'Address', 'Location type', 'dest_type', 'Latitude',
       'Longitude'],
      dtype='object')

,Location,Address,Location type,dest_type,Latitude,Longitude
0,DeVry University,"3505 Koger Blvd #100, Duluth, GA 30096",College Campus-Potential,potential,33.960284,-84.124450
1,Georgia Gwinnett College,1000 University Center Ln,College Campus-Potential,potential,33.981360,-84.000130
2,Gwinnett College - Lilburn Campus,"4230 US-29 #11, Lilburn, GA 30047",College Campus-Potential,potential,33.915850,-84.120330
3,Gwinnett Technical College,"5150 Sugarloaf Pkwy, Lawrenceville, GA 30043",College Campus-Potential,potential,33.974521,-84.068832
4,UGA Gwinnett Campus,"2530 Sever Rd NW, Lawrenceville, GA 30043",College Campus-Potential,potential,34.010387,-84.072952


Index(['Location', 'Latitude', 'Longitude', 'Address', 'Location type',
       'dest_type'],
      dtype='object')

array(['potential', 'polling', 'bg_centroid'], dtype=object)

In [9]:
#cross join polling location data with the census data, and compute straight line distances

full_df = combined_df.merge(all_locations, how= 'cross')

full_df['distance_m'] = full_df.apply(lambda row: haversine((row.INTPTLAT20, row.INTPTLON20), (row.Latitude, row.Longitude)), axis=1)*1000
full_df.shape

(4221514, 22)

In [11]:
full_df.columns

Index(['GEO_ID', 'NAME', 'hispanic', 'non-hispanic', 'population', 'white',
       'black', 'native', 'asian', 'pacific_islander', 'other',
       'multiple_races', 'GEOID20', 'INTPTLAT20', 'INTPTLON20', 'Location',
       'Address', 'Location type', 'dest_type', 'Latitude', 'Longitude',
       'distance_m'],
      dtype='object')

In [10]:
#full_df[['INTPTLAT20', 'INTPTLON20', 'Location', 'Address', 'Latitude', 'Longitude', 'distances_m']].head()
#prepare data for writing

full_df = full_df.rename(columns = {'GEO_ID': 'id_orig', 'Address': 'address', 'Latitude':'dest_lat', 'Longitude':'dest_lon', 'INTPTLAT20':'orig_lat', 'INTPTLON20':'orig_lon', 'Location type': 'location_type', 'Location': 'id_dest'})
full_df['county'] = 'Gwinnett_GA'

FULL_DF_COLS = [
    'id_orig',
    'id_dest',
    'distance_m',
    'county',
    'address',
    'dest_lat',
    'dest_lon',
    'orig_lat',
    'orig_lon',
    'location_type',
    'dest_type',
    'population',
    'hispanic', 
    'non-hispanic',
    'white', 
    'black', 
    'native', 
    'asian',
    'pacific_islander', 
    'other', 
    'multiple_races',
]

full_df = full_df[FULL_DF_COLS]

full_df.shape

(4221514, 21)

In [11]:
#write to file
full_df.to_csv('datasets/Gwinnett_GA.csv', index = True)


### Once a run is complete, the following cell will geotag the matched locations for mapping purposes.
### DO NOT run as part of the source build. 

In [73]:
GEOTAGGED_SOURCE_FILE = 'datasets/Gwinnett_GA.csv'
EA_OUTPUT_FILE = 'Gwinnett_result/Gwinnett_[2020, 2022]_full_beta=-2_min_old=0_max_new=1_num_locations=50_precinct_distances.csv'

ea_output = pd.read_csv(EA_OUTPUT_FILE)
ea_input = pd.read_csv(GEOTAGGED_SOURCE_FILE)

ea_output_unique = ea_output.loc[ea_output.demographic == 'population']

ea_input['Lat, Long'] = ea_input.dest_lat.astype(str)+', ' +ea_input.dest_lon.astype(str)
ea_input_relevant = ea_input[['id_dest', 'Lat, Long']]
ea_input_relevant = ea_input_relevant.drop_duplicates()

geotagged_output = ea_output_unique.merge(ea_input_relevant, on = 'id_dest', how = 'left')

#write to file
geotagged_output.to_csv('Gwinnett_result/Gwinnett_[2020, 2022]_full_beta=-2_min_old=0_max_new=1_num_locations=50_precinct_geotagged.csv', index = True)

AttributeError: 'str' object has no attribute 'dtype'